# Install and Setup Kubeflow on EKS

Based on these instructions:  https://www.kubeflow.org/docs/aws/deploy/install-kubeflow/

# _This Will Take a Few Minutes, Please Be Patient._

In [ ]:
%%bash

source ~/.bash_profile

#### Check environment variables are set with valid values
echo $S3_BUCKET
echo $AWS_CLUSTER_NAME
echo $INSTANCE_ROLE_NAME
echo $INSTANCE_PROFILE_ARN

#### Download the `kfctl` CLI tool
curl --location https://github.com/kubeflow/kfctl/releases/download/v1.0.2/kfctl_v1.0.2-0-ga476281_linux.tar.gz | tar xz
sudo mv kfctl /usr/local/bin

#### Get the latest Kubeflow configuration file

# With Ingress
export CONFIG_URI="https://raw.githubusercontent.com/kubeflow/manifests/v1.0-branch/kfdef/kfctl_aws.v1.0.2.yaml"
#export CONFIG_URI='https://raw.githubusercontent.com/kubeflow/manifests/v1.0.2/kfdef/kfctl_aws.v1.0.2.yaml'
echo "export CONFIG_URI=${CONFIG_URI}" | tee -a ~/.bash_profile

#### Set Kubeflow environment variables 
export KF_NAME=${AWS_CLUSTER_NAME}
echo "export KF_NAME=${KF_NAME}" | tee -a ~/.bash_profile

#cd ~/SageMaker/kubeflow/notebooks/part-3-kubernetes

export KF_DIR=$PWD/${KF_NAME}
echo "export KF_DIR=${KF_DIR}" | tee -a ~/.bash_profile

#### Customize the configuration files
# We'll edit the configuration with the right names for the cluster and node groups before deploying Kubeflow.

mkdir -p ${KF_DIR}
cd ${KF_DIR}

curl -O ${CONFIG_URI}

export CONFIG_FILE=${KF_DIR}/kfctl_aws.v1.0.2.yaml
echo "export CONFIG_FILE=${CONFIG_FILE}" | tee -a ~/.bash_profile

sed -i.bak -e "/region: us-west-2/ a \      enablePodIamPolicy: true" ${CONFIG_FILE}
sed -i.bak -e "s@us-west-2@$AWS_REGION@" ${CONFIG_FILE}
sed -i.bak -e "s@roles:@#roles:@" ${CONFIG_FILE}
sed -i.bak -e "s@- eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@#- eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@" ${CONFIG_FILE}
sed -i.bak -e "s@eksctl-kubeflow-aws-nodegroup-ng-a2-NodeInstanceRole-xxxxxxx@$INSTANCE_ROLE_NAME@" ${CONFIG_FILE}
sed -i.bak -e 's/kubeflow-aws/'"$AWS_CLUSTER_NAME"'/' ${CONFIG_FILE}

#### Generate the Kubeflow installation files
cd ${KF_DIR}

rm -rf kustomize
rm -rf .cache

kfctl build -V -f ${CONFIG_FILE}

#### Deploy Kubeflow
cd ${KF_DIR}

kfctl apply -V -f ${CONFIG_FILE}

#### Remove istio-ingress in --namespace kubeflow
kubectl delete ingress istio-ingress --namespace kubeflow

#### Delete the usage reporting beacon
kubectl delete all -l app=spartakus --namespace=kubeflow

#### Change to `kubeflow` namespace
kubectl config set-context --current --namespace=kubeflow

## Wait for resource to become available
Monitor changes by running kubectl get all namespaces command.


In [ ]:
!kubectl -n kubeflow get all


In [ ]:
!kubectl -n istio-system get all


In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();